## Imports

In [1]:
DEV = True

import pandas as pd
import numpy as np

import os
import json
import time
import datetime

from os import listdir
from os.path import isfile
from tqdm import tqdm_notebook as tqdm

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

import lightgbm as lgb
import catboost as cb

from sklearn.model_selection import StratifiedKFold, train_test_split, ParameterSampler
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import fbeta_score, cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.externals import joblib
from sklearn.preprocessing import scale

from bayes_opt import BayesianOptimization
from bayes_opt.observer import JSONLogger
from bayes_opt.event import Events
from bayes_opt.util import load_logs

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
input_path = "../input/"
output_path = "../output/"
model_path = "../models/"

ensure_dir(model_path)
ensure_dir(output_path)

## Read data

In [2]:
train_df = pd.read_csv(input_path+'train.csv.zip')

label = train_df.target
train = train_df.drop(['ID_code','target'],axis=1)

test = pd.read_csv(input_path+'test.csv.zip')
test = test.drop(['ID_code'],axis=1)

test_filtered = pd.read_pickle(input_path+'test_filtered.pkl')
test_filtered = test_filtered.loc[:,train.columns]

train_test = pd.concat([train,test_filtered]).reset_index(drop=True)

In [48]:
vcs_train_test = {}
vcs_train_test_3 = {}

for i in tqdm(range(200)):
    col = "var_"+str(i)
    tmp = train_test.loc[:,col].apply(lambda x: round(x,3))
    vcs_train_test[col] = train_test.loc[:,col].value_counts()
    vcs_train_test_3[col] = tmp.value_counts()# /300000
#     vtraintest = vcs_train_test[col]
#     t = vtraintest[train_test[col]].values
#     train_test[col+'_train_test_sum_vcs'] = t
#     train_test[col+'_train_test_sum_vcs_prod'] = train_test[col]*t
#     vcs_train_test_prod[col] = train_test.loc[:,col+'_train_test_sum_vcs_prod'].value_counts()

In [51]:
def feature_generator(df):
    for i in tqdm(range(200)):
        col = "var_"+str(i)
        
        vtraintest = vcs_train_test[col]
        t = vtraintest[df[col]].values #fillna(0).values
        df[col+'_train_test_sum_vcs'] = t
        df[col+'_train_test_sum_vcs_prod'] = df[col]*t
        
        vtraintest = vcs_train_test_3[col]
        t = vtraintest[df[col].apply(lambda x: round(x,3))].values #fillna(0).values
        df[col+'_train_test_sum_vcs_3'] = t
        df[col+'_train_test_sum_vcs_prod_3'] = df[col].apply(lambda x: round(x,3))*t
        
#         vtraintest_prod = vcs_train_test_prod[col]
#         t = vtraintest_prod[df[col]].values #fillna(0).values
#         df[col+'_train_test_sum_vcs_'] = t
#         df[col+'_train_test_sum_vcs_prod_'] = df[col]*t
        
#         df[col+'_train_test_sum_vcs_sign'] = (df[col+"_train_test_sum_vcs_prod"]>0).astype(int)
#         df[col+'_train_test_sum_vcs_div'] = df[col]/t
#         df[col+'_train_test_sum_vcs_minus'] = scale(df[col]) - scale(t)
#         df[col+'_train_test_sum_vcs_plus'] = scale(df[col]) + scale(t)
#         df[col+'_train_test_sum_vcs_min'] = np.min(scale(df[col]), scale(t))
#         df[col+'_train_test_sum_vcs_max'] = np.max(scale(df[col]), scale(t))
#         df[col+'_train_test_sum_vcs_pow'] = np.power(abs(scale(df[col])), abs(scale(t)))
#         df[col+'_train_test_sum_vcs_log'] = np.log(abs(scale(df[col])), abs(scale(t)))
        
feature_generator(train)
feature_generator(test)

In [52]:
df = train
dt = test

cols = sorted(list(set(df.columns)-{"label"}))
print(len(cols))

df["label"] = label
df.label.value_counts()

1001


0    179902
1     20098
Name: label, dtype: int64

In [53]:
train.var_34_train_test_sum_vcs_prod_3.value_counts()



2212.056    255
2729.290    174
2630.294    169
2746.520    167
2539.890    167
2771.922    163
2699.305    163
2622.296    163
2652.239    162
2524.032    162
2667.151    161
2693.247    160
2662.584    159
2620.695    159
2768.849    158
2454.898    158
2609.152    158
2644.568    158
2748.394    158
2596.631    157
2563.385    156
2262.029    156
2555.411    156
2551.003    156
2564.946    155
2593.475    155
2581.225    155
2594.932    154
2402.610    154
2636.136    154
           ... 
12.768        1
38.715        1
38.730        1
20.084        1
13.057        1
12.922        1
9.922         1
12.974        1
26.038        1
10.017        1
19.948        1
13.160        1
13.040        1
20.022        1
60.198        1
20.154        1
9.975         1
12.970        1
12.904        1
12.863        1
12.856        1
19.920        1
9.842         1
12.877        1
30.015        1
9.880         1
12.900        1
12.980        1
12.827        1
20.118        1
Name: var_34_train_test_

## Scaling

In [ ]:
scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = scaler.fit_transform(df[cols])
Z = scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Logistic regression

In [ ]:
%%time

df["clf"] = 0
dt["clf"] = 0

i = -1
for train_index, valid_index in skf.split(X, y):
    i+=1
        
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]
    
    fname = model_path+'clf_'+str(i)+'.pkl'
    if DEV or not isfile(fname):
        c = 1
        clf = LogisticRegression(C=c,
                                 solver="newton-cg",
                                 penalty="l2", 
                                 n_jobs=-1, 
                                 max_iter=1000).fit(X_train, y_train) 
        joblib.dump(clf, fname)
    else:
        clf = joblib.load(fname)

    y_pred = clf.predict_proba(X_valid)[:,1] 
    df.loc[valid_index, "clf"] = y_pred
    print("ROC AUC:", round(auc(y_valid, y_pred), 4), 
          "\nF-score:", round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), i)
    
    dt["clf"] += clf.predict(Z)/5
    

print("\nROC AUC:", round(auc(df.label, df["clf"]), 4), 
      "\nF-score:", round(fbeta_score(df.label, df["clf"]>0.5, beta=1), 4))

## SVM

In [ ]:
#cols+=["clf"]

scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = scaler.fit_transform(df[cols])
Z = scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
%%time

df["svc"] = 0
dt["svc"] = 0

i = -1
for train_index, valid_index in skf.split(X, y):
    i+=1
        
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]
    
    fname = model_path+'svc_'+str(i)+'.pkl'
    #print(fname)
    if DEV or not isfile(fname):
        c = 100
        svc = SVC(C=c,
                  probability=True).fit(X_train, y_train) 
        
        joblib.dump(svc, fname)
    else:
        svc = joblib.load(fname)

    y_pred = svc.predict_proba(X_valid)[:,1] 
    df.loc[valid_index, "svc"] = y_pred
    print("ROC AUC:", round(auc(y_valid, y_pred), 4), 
          "\nF-score:", round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), i)
    
    dt["svc"] += svc.predict(Z)/5


print("\nROC AUC:", round(auc(df.label, df["svc"]), 4), 
      "\nF-score:", round(fbeta_score(df.label, df["svc"]>0.5, beta=1), 4))

## Nearest Neighbors

In [ ]:
#cols+=["svc"]

scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = scaler.fit_transform(df[cols])
Z = scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
%%time

df["nei"] = 0
dt["nei"] = 0

i = -1
for train_index, valid_index in skf.split(X, y):
    i+=1
        
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]
    
    fname = model_path+'nei_'+str(i)+'.pkl'
    if DEV or not isfile(fname):
        nei = KNeighborsClassifier(n_neighbors=50,
                                   p=1, n_jobs=-1).fit(X_train, y_train) 
        joblib.dump(nei, fname)
    else:
        nei = joblib.load(fname)

    y_pred = nei.predict_proba(X_valid)[:,1] 
    df.loc[valid_index, "nei"] = y_pred
    print("ROC AUC:", round(auc(y_valid, y_pred), 4), 
          "\nF-score:", round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), i)
    
    dt["nei"] += nei.predict(Z)/5
    

print("\nROC AUC:", round(auc(df.label, df["nei"]), 4), 
      "\nF-score:", round(fbeta_score(df.label, df["nei"]>0.5, beta=1), 4))

## LighGBM

In [54]:
#cols+=["nei"]

# k = 0
# cols = [f"var_{k}",
#         f"var_{k}_train_test_sum_vcs",
#         f"var_{k}_train_test_sum_vcs_prod"]

scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = scaler.fit_transform(df[cols])
#Z = dt[cols # ]scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [55]:
rounds = 10000
early_stop_rounds = 200

params = {'lambda_l1': 0, 
          'lambda_l2': 0,
          'feature_fraction':0.9,
#           'bagging_fraction':0.1,
#           'bagging_freq': 1,
          'learning_rate': 0.03, 
          'max_depth': 4,
          #'min_data_in_leaf':20,
          #'num_leaves':2**5-1,
          'boosting_type': 'gbrt', #dart gbrt
          'objective': 'binary', 
          'metric': 'auc',
          #'weight': [1, 0.5],
          #'device': 'gpu',
          #'gpu_platform_id': '0',
          #'gpu_device_id': '0',
          'max_bin': 1024,
          'n_jobs':-1
         }

In [56]:
%%time

df["lgb"] = 0
dt["lgb"] = 0

i = -1
for train_index, valid_index in skf.split(X, y):
    i+=1
        
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]

    d_train = lgb.Dataset(X_train, y_train, feature_name=cols)
    d_valid = lgb.Dataset(X_valid, y_valid, feature_name=cols)    
    
    fname = model_path+'lgb_'+str(i)+'.pkl'
    if DEV or not isfile(fname):
        model = lgb.train(params,
                            d_train,
                            num_boost_round=rounds,
                            valid_sets=[d_train, d_valid],
                            valid_names=['train','valid'],
                            #feval=lgb_f1_score,
                            early_stopping_rounds=early_stop_rounds,
                            verbose_eval=100)
        
        joblib.dump(model, fname)
    else:
        model = joblib.load(fname)
    
    
    y_pred = model.predict(X_valid)
    df.loc[valid_index, "lgb"] = y_pred
    print("ROC AUC:", round(auc(y_valid, y_pred), 4), 
      "\nF-score:", round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), i)
    
    #dt["lgb"] += model.predict(Z)/5
    break
    
# print("\nROC AUC:", round(auc(df.label, df["lgb"]), 4), 
#       "\nF-score:", round(fbeta_score(df.label, df["lgb"]>0.5, beta=1), 4))

# [3992]	train's auc: 0.989264	valid's auc: 0.913532
# ROC AUC: 0.9135 
# F-score: 0.5195 0

Training until validation scores don't improve for 200 rounds.
[100]	train's auc: 0.842046	valid's auc: 0.809278
[200]	train's auc: 0.88315	valid's auc: 0.845162
[300]	train's auc: 0.90458	valid's auc: 0.863268
[400]	train's auc: 0.918961	valid's auc: 0.87402
[500]	train's auc: 0.929144	valid's auc: 0.881456
[600]	train's auc: 0.937068	valid's auc: 0.886814
[700]	train's auc: 0.94321	valid's auc: 0.891072
[800]	train's auc: 0.948354	valid's auc: 0.894293
[900]	train's auc: 0.952496	valid's auc: 0.896859
[1000]	train's auc: 0.956152	valid's auc: 0.898978
[1100]	train's auc: 0.959277	valid's auc: 0.900682
[1200]	train's auc: 0.962134	valid's auc: 0.90222
[1300]	train's auc: 0.964719	valid's auc: 0.903434
[1400]	train's auc: 0.967117	valid's auc: 0.904579
[1500]	train's auc: 0.969281	valid's auc: 0.905521
[1600]	train's auc: 0.97123	valid's auc: 0.906321
[1700]	train's auc: 0.973155	valid's auc: 0.907039
[1800]	train's auc: 0.974857	valid's auc: 0.907663
[1900]	train's auc: 0.976385	valid

In [23]:
df.loc[valid_index, "lgb"][df.label==1][df.lgb>0.1]

13        0.121532
84        0.113602
525       0.102129
577       0.141882
1312      0.101255
1423      0.105894
1490      0.125100
1858      0.160463
1863      0.107850
2093      0.115804
2118      0.101529
2214      0.107850
2301      0.103038
2392      0.106306
2682      0.128267
2684      0.149859
2718      0.102518
2827      0.116290
3134      0.255105
3219      0.206279
3388      0.141882
3396      0.123623
3444      0.239903
3842      0.107250
3986      0.107850
4103      0.112789
4251      0.154898
4333      0.206279
4600      0.111623
4940      0.246335
            ...   
196368    0.103798
196577    0.135669
196700    0.110354
196771    0.106646
196803    0.226802
196806    0.161413
197057    0.129187
197513    0.113602
197752    0.107044
197926    0.118239
198089    0.118239
198250    0.116290
198584    0.116535
198643    0.113762
198689    0.110793
198733    0.113840
198815    0.105201
198916    0.136829
198929    0.113416
199023    0.125083
199103    0.161789
199131    0.

In [ ]:
w = [1,2,5]

df["avg"] = (w[0]*df["clf"]+w[1]*df["svc"]+w[2]*df["lgb"])/sum(w)
dt["avg"] = (w[0]*dt["clf"]+w[1]*dt["svc"]+w[2]*dt["lgb"])/sum(w)

print("\nROC AUC:", round(auc(df.label, df["avg"]), 4), 
      "\nF-score:", round(fbeta_score(df.label, df["avg"]>0.5, beta=1), 4))

In [ ]:
np.sum(dt["avg"] > 0.5) / len(dt), np.sum(df["avg"] > 0.5) / len(df)

In [ ]:
predictions = list(dt["avg"] > 0.5)
np.mean(predictions[:2*len(predictions)//5])

In [ ]:
dt[["Image","avg"]].to_csv("test_prediction_top5.csv", index=False)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 300))
lgb.plot_importance(model, max_num_features=len(cols), ax=ax)
plt.title("Light GBM Feature Importance")

## False positive

In [ ]:
df[(df["lgb"]>0.5)&(df.label!=1)]

## False negative

In [ ]:
df[(df["lgb"]<0.5)&(df.label==1)]

## Correlation

In [ ]:
df.corr().label

## HPO

In [ ]:
scaler = StandardScaler() #MinMaxScaler StandardScaler RobustScaler

X = scaler.fit_transform(df[cols])
Z = scaler.transform(dt[cols])
y = list(df.label)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
param_grid = {  
              'max_depth': [3,4,5],
              'learning_rate': np.linspace(0.01, 0.1, 10),
              'feature_fraction': np.linspace(0.5, 0.9, 5),
              'lambda_l1': np.linspace(1, 10, 10),
              'lambda_l2': np.linspace(1, 10, 10)
            }

param_static = {
                'boosting_type': 'gbrt',
                'objective': 'binary',
                'metric': 'auc',
                'max_bin': 1024,
                'n_jobs': -1
               }

param_list = list(ParameterSampler(param_grid, 
                                   n_iter=10))

rounded_list = [dict((k, round(v, 6)) for (k, v) in d.items())
                for d in param_list]

rounds = 10000
early_stop_rounds = 200

In [ ]:
for train_index, valid_index in skf.split(X, y):
    X_train = X[train_index, :]
    X_valid = X[valid_index, :]

    y_train = df.loc[train_index, "label"]
    y_valid = df.loc[valid_index, "label"]

    d_train = lgb.Dataset(X_train, y_train, feature_name=cols)
    d_valid = lgb.Dataset(X_valid, y_valid, feature_name=cols)    
    
    break

results = []
for i, params in enumerate(rounded_list):
    print(params)
    start_time = datetime.datetime.fromtimestamp(time.time())
    
    for key in param_static:
        params[key] = param_static[key]
    
    model = lgb.train(params,
                      d_train,
                      num_boost_round=rounds,
                      valid_sets=[d_train, d_valid],
                      valid_names=['train','valid'],
                      #feval=lgb_f1_score,
                      early_stopping_rounds=early_stop_rounds,
                      verbose_eval=0)

    y_pred = model.predict(X_valid)
    
    end_time = datetime.datetime.fromtimestamp(time.time())
    seconds = (end_time - start_time).total_seconds()
    minutes = round(seconds/60, 1)
    
    res = auc(y_valid, y_pred)
    print(i, "ROC AUC:", round(res, 4), "F-score:", 
          round(fbeta_score(y_valid, y_pred>0.5, beta=1), 4), 
          minutes, "minutes\n")
    
    results.append((params, res))
    for key in param_static:
        del params[key]

In [ ]:
sorted_results = sorted(results, key=lambda x: x[1], reverse=True)
sorted_results[0]